# 0. Import libraries

In [ ]:
import os
import sys
import pypsa
import pandas as pd
import geopandas as gpd
import pycountry
import logging
import warnings
warnings.filterwarnings("ignore")

# 1. Define functions

In [ ]:
def create_folder(directory):
    """ Input:
            directory - path to directory
        Output:
            N/A
    """
    # Create the folder if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Folder '{directory}' created.")
    else:
        print(f"Folder '{directory}' already exists.")


def get_country_list(directory):
    """ Input:
            directory - path to results directory
        Output:
            List of countries in two-letter format
    """
    filenames = os.listdir(directory)
    country_names = set([x[:2] for x in filenames])
    return sorted(list(country_names))


def get_network_path(country_code, horizon, directory):
    """ Input:
            country_code - two-letter country code
            horizon - planning horizon
            directory - path to results directory
        Output:
            filepath - full path to network
    """
    foldername = f"{country_code}_{horizon}/networks"
    filenames = os.listdir(os.path.join(directory, foldername))
    if len(filenames) != 1:
        logging.warning(f"Only 1 network per scenario is allowed currently!")
    filepath = os.path.join(directory, foldername, filenames[0])   
    return filepath


def get_base_network_path(country_code, horizon, directory):
    """ Input:
            country_code - two-letter country code
            horizon - planning horizon
            directory - path to results directory
        Output:
            filepath - full path to base network
    """
    foldername = f"{country_code}_{horizon}/"
    filenames = os.listdir(os.path.join(directory, foldername))

    filepath = os.path.join(directory, foldername, "base.nc")
    return filepath


def load_network(filepath):
    """ Input:
            filepath - full path to the network
        Output:
            n - PyPSA network
    """
    try:
        n = pypsa.Network(filepath)
        logging.info(f"Loading {filepath}")
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None
    return n


def get_total_demand(n):
    """ Input:
            n - network
        Output:
            demand - total electricity demand in TWh
    """
    demand = n.loads_t.p_set.multiply(n.snapshot_weightings.objective, axis=0).sum().sum() / 1e6
    return demand.round(4)


def get_installed_capacities(n):
    """ Input:
            n - network
        Output:
            capacities - total electricity generation capacities in GW
    """
    # generator capacities
    gen_capacities = n.generators.groupby("carrier").p_nom.sum()
    # storage unit capacities
    storage_capacities = n.storage_units.groupby("carrier").p_nom.sum()
    # total capacities
    capacities = (pd.concat([gen_capacities, storage_capacities], axis=0) / 1e3).round(4)
    # drop load shedding
    if "load" in n.generators.carrier.unique():
        capacities.drop("load", inplace=True)
    return capacities


def get_generation_mix(n):
    """ Input:
            n - network
        Output:
            generation_mix - amount of consumed electricity generated by different carrier in TWh
    """
    # electricity generation by generators
    gen_generation = n.generators_t.p.multiply(n.snapshot_weightings.objective, axis=0).groupby(n.generators.carrier, axis=1).sum().sum()
    # electricity generation by storage unit
    storage_generation = n.storage_units_t.p.multiply(n.snapshot_weightings.objective, axis=0).groupby(n.storage_units.carrier, axis=1).sum().sum()
    # total generation mix
    generation_mix = pd.concat([gen_generation, storage_generation], axis=0) / 1e6
    return generation_mix.round(4)


def get_country_name(country_code):
    """ Input:
            country_code - two letter code of the country
        Output:
            country.name - corresponding name of the country
            country.alpha_3 - three letter code of the country
    """
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        return country.name, country.alpha_3 if country else None
    except KeyError:
        return None


def load_ember_data():
    """
    Load the Ember data from a CSV file located in the data folder.

    Returns:
        pd.DataFrame: Ember data in long format.
    """
    ember_data_path = os.path.join(os.getcwd(), "../data/ember_yearly_full_release_long_format.csv")
    ember_data = pd.read_csv(ember_data_path)
    return ember_data


def get_eia_historic_demand(country_code, horizon):
    """ Input:
            country_code - two letter code of the country
        Output:
            demand - electricity demand in TWh from EIA (https://www.eia.gov/international/data/world/electricity/electricity-consumption)
    """
    # load EIA data
    eia_data = pd.read_csv(DATA_DIR+"EIA_demands.csv")
    eia_data.rename(columns={"Unnamed: 1":"country"}, inplace=True)
    eia_data["country"] = eia_data["country"].str.strip()
    eia_data.dropna(subset=['API'], inplace=True)
    eia_data["code_3"] = eia_data["API"].apply(lambda x: x.split('-')[2] if isinstance(x, str) and len(x.split('-'))>3 else x)
    
    # get the name of the country and three letter code
    country_name, country_code3 = get_country_name(country_code)
    
    # select the demand for given country and year in TWh (select based on 3 letter code first, then country name)
    if country_code3 and country_code3 in eia_data.code_3.unique():
        demand = float(eia_data.query("code_3 == @country_code3")[str(horizon)].values[0])
    elif country_name and country_name in eia_data.country.unique():
        demand = flaot(eia_data.query("country == @country_name")[str(horizon)].values[0])
    else:
        demand = None
    return demand


def get_ember_historic_demand(country_code, horizon):
    """
    Get the electricity demand for a given country and year from Ember data.

    Args:
        country_code (str): Country code (ISO 3166-1 alpha-2).
        year (int): Year of interest.

    Returns:
        float or None: Electricity demand if found, otherwise None.
    """
    # Load Ember data
    data = load_ember_data()
    # get the name of the country and three letter code
    country_name, country_code3 = get_country_name(country_code)
    
    demand = data[(data["Year"] == horizon) 
                  & (data["Country code"] == country_code3) 
                  & (data["Category"] == "Electricity demand") 
                  & (data["Subcategory"] == "Demand")]["Value"]

    if len(demand) != 0:
        return demand.iloc[0]
    return None


def get_eia_historic_capacities(country_code, horizon):
    """ Input:
            country_code - two letter code of the country
        Output:
            capacity - generation capacities in MW from EIA (https://www.eia.gov/international/data/world/electricity/electricity-capacity)
    """
    # load EIA data
    eia_data = pd.read_csv(DATA_DIR+"EIA_installed_capacities.csv")
    eia_data.rename(columns={"Unnamed: 1":"country"}, inplace=True)
    eia_data["country"] = eia_data["country"].str.strip()
    eia_data["code_3"] = eia_data.dropna(subset=["API"])["API"].apply(lambda x: x.split('-')[2] if isinstance(x, str) and len(x.split('-'))>3 else x)

    # get the name of the country and three letter code
    country_name, country_code3 = get_country_name(country_code)

    # find capacities for given country
    if country_code3 and country_code3 in eia_data.code_3.unique():
        capacities = eia_data.query("code_3 == @country_code3")[["country", str(horizon)]]
    elif country_name and country_name in eia_data.country.unique():
        country_index = eia_data.query("country == @country_name").index[0]
        capacities = eia_data.iloc[country_index+1:country_index+14][["country", str(horizon)]]
    else:
        capacities = None
    return capacities


def get_ember_historic_capacities(country_code, horizon):
    """
    Get installed capacity by fuel type for a given country and year from Ember data.

    Args:
        country_code (str): Country code (ISO 3166-1 alpha-2).
        horizon (int): Year of interest.

    Returns:
        pd.DataFrame: Installed capacity by fuel type.
    """
    # Load Ember data
    data = load_ember_data()
    # get the name of the country and three letter code
    country_name, country_code3 = get_country_name(country_code)
    
    capacity_ember = data[
        (data["Country code"] == country_code3) 
        & (data["Year"] == horizon)
        & (data["Category"] == "Capacity") 
        & (data["Subcategory"] == "Fuel")][["Variable", "Value"]].reset_index(drop=True)
    
    # Drop irrelevant rows
    drop_row = ["Other Renewables"]
    capacity_ember = capacity_ember[~capacity_ember["Variable"].isin(drop_row)]

    # Standardize fuel types
    capacity_ember = capacity_ember.replace({
        "Gas": "Fossil fuels", 
        "Bioenergy": "Biomass", 
        "Coal": "Fossil fuels",
        "Other Fossil": "Fossil fuels"})
    
    capacity_ember = capacity_ember.groupby("Variable").sum()
    capacity_ember.columns = ["Ember data"]

    return capacity_ember


def get_eia_historic_generation(country_code, horizon):
    """ Input:
            country_code - two letter code of the country
        Output:
            generation  - generation capacities in TWh from EIA (https://www.eia.gov/international/data/world/electricity/electricity-generation)
    """
    # load EIA data
    eia_data = pd.read_csv(DATA_DIR+"EIA_electricity_generation.csv")
    eia_data.rename(columns={"Unnamed: 1":"country"}, inplace=True)
    eia_data["country"] = eia_data["country"].str.strip()
    eia_data["code_3"] = eia_data.dropna(subset=["API"])["API"].apply(lambda x: x.split('-')[2] if isinstance(x, str) and len(x.split('-'))>3 else x)
    
    # get the name of the country and three letter code
    country_name, country_code3 = get_country_name(country_code)
    
    # find generation values for given country
    if country_code3 and country_code3 in eia_data.code_3.unique():
        generation = eia_data.query("code_3 == @country_code3")[["country", str(horizon)]]
    elif country_name and country_name in eia_data.country.unique():
        country_index = eia_data.query("country == @country_name").index[0]
        generation = eia_data.iloc[country_index+1:country_index+18][["country", str(horizon)]]
    else:
        generation = None
    return generation


def get_ember_historic_generation(country_code, horizon):
    """
    Get electricity generation by fuel type for a given country and year from Ember data.

    Args:
        country_code (str): Country code (ISO 3166-1 alpha-2).
        horizon (int): Year of interest.

    Returns:
        pd.DataFrame: Electricity generation by fuel type.
    """
    # Load Ember data
    data = load_ember_data()
    # get the name of the country and three letter code
    country_name, country_code3 = get_country_name(country_code)
    
    generation_ember = data[
        (data["Category"] == "Electricity generation")
        & (data["Country code"] == country_code3)
        & (data["Year"] == horizon)
        & (data["Subcategory"] == "Fuel")
        & (data["Unit"] == "TWh")
            ][["Variable", "Value"]].reset_index(drop=True)

    # Drop irrelevant rows
    drop_row = ["Other Renewables"]
    generation_ember = generation_ember[~generation_ember["Variable"].isin(drop_row)]

    # Standardize fuel types
    generation_ember = generation_ember.replace({
        "Gas": "Natural gas", 
        "Bioenergy": "Biomass", 
        "Other Fossil": "Oil"})
    
    # Group by fuel type
    generation_ember = generation_ember.groupby("Variable").sum()
    generation_ember.loc["Load shedding"] = 0.0
    generation_ember.columns = ["Ember data"]

    return generation_ember


def get_network_length(n_base):
    """ Input:
            n_base - base network of the country
        Output:
            length - total network length in km
            voltage_ratings - voltage ratings in kV
    """
    length = n_base.lines.length.sum().astype(int)
    voltage_ratings = n_base.lines.v_nom.astype(int).sort_values().unique()
    return length, voltage_ratings
    

def real_network_length(country_code):
    """ Input:
            country_code - two letter code of the country
        Output:
            length - total network length in km
            voltage_ratings - voltage ratings in kV
    """
    line_length = {"AU":56192, "BR":179297, "CN":1604838, "CO":29169, 
                   "DE":35796, "IN":706348, "IT":75246, "MX":109747, 
                   "NG":25633, "US":682812, "ZA": 0} # Eg: https://www.power-technology.com/data-insights/top-five-transmission-line-projects-in-colombia/
    voltage_ratings = {"AU": [66, 132, 275, 330], "BR": [138, 230, 345, 440, 500, 600, 750, 800], # https://ri.taesa.com.br/en/taesa/transmission-sector/ 
                       "CN": [110, 220, 330, 500, 750, 800, 1000], "CO": [110, 115, 220, 230, 500], # https://www.cigre.org/userfiles/files/Community/National%20Power%20System/2020_National_power_system_COLOMBIA%20-%202020_National_power_system_COLOMBIA.pdf
                       "DE": [110, 220, 380], "IN": [66, 132, 220, 400, 765], 
                       "IT": [132, 150, 220, 380], "MX": [115, 230, 400], "NG": [132, 330],
                       "US": [69, 115, 138, 230, 345, 500, 765], "ZA": [66, 88, 132, 220, 330]}
    return line_length[country_code], voltage_ratings[country_code]

    
def compare_capacities(network_capacities, eia_historic_capacities, ember_historic_capacities):
    """ Input:
            network_capacities - capacities obtained from the network in GW
            eia_historic_capacities - capacities obtained from EIA in GW
            ember_historic_capacities - capacities obtained from Ember in GW
        Output:
            capacities - DataFrame comparing two data
    """
    # bring EIA historic data into format
    eia_historic_capacities["country"] = eia_historic_capacities.country.str.replace("(million kW)","").str.strip()
    eia_historic_capacities.set_index("country", inplace=True)
    eia_historic_capacities.columns = ["EIA"]
    eia_historic_capacities.rename(index={"Capacity":"Total capacity", 
                                          "Hydroelectricity":"Hydro", 
                                          "Biomass and waste":"Biomass", 
                                          "Hydroelectric pumped storage":"PHS"}, inplace=True)
    eia_historic_capacities.drop(index=["Renewables", "Non-hydroelectric renewables", 
                                        "Solar, tide, wave, fuel cell", "Tide and wave"], inplace=True)
    eia_historic_capacities = eia_historic_capacities.loc[["Nuclear", "Fossil fuels", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Geothermal", "Total capacity"], :]

    # bring Ember historic data into format
    ember_historic_capacities.columns = ["Ember"]
    ember_historic_capacities.loc["Total capacity", :] = ember_historic_capacities.sum()
    
    # bring capacities from the network into format
    all_carriers = ["nuclear", "coal", "lignite", "CCGT", "OCGT", "hydro", "ror", "PHS", "solar", "offwind-ac", "offwind-dc", "onwind", "biomass", "geothermal"]
    network_capacities = network_capacities.reindex(all_carriers, fill_value=0)
    network_capacities.rename(index={"nuclear":"Nuclear", "solar":"Solar", "biomass":"Biomass", "geothermal":"Geothermal"}, inplace=True)
    network_capacities["Fossil fuels"] = network_capacities[["coal", "lignite", "CCGT", "OCGT"]].sum()
    network_capacities["Hydro"] = network_capacities[["hydro", "ror"]].sum()
    network_capacities["Wind"] = network_capacities[["offwind-ac", "offwind-dc", "onwind"]].sum()
    network_capacities = network_capacities.loc[["Nuclear", "Fossil fuels", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Geothermal"]]
    network_capacities["Total capacity"] = network_capacities.sum()
    network_capacities.name = "PyPSA Model"
    network_capacities = network_capacities.to_frame()

    # merge two data
    capacities = pd.concat([network_capacities, eia_historic_capacities, ember_historic_capacities], axis=1).astype(float)
    capacities["Error wrt EIA (%)"] = (100*(capacities["PyPSA Model"] - capacities["EIA"])/capacities["EIA"]).astype(float).round(2)
    capacities["Error wrt Ember (%)"] = (100*(capacities["PyPSA Model"] - capacities["Ember"])/capacities["Ember"]).astype(float).round(2)
    capacities.fillna(0, inplace=True)
    capacities.index.name = "Capacities [GW]"
    return capacities.round(2)


def compare_demands(network_demand, eia_historic_demand, eia_ember_demands):
    """ Input:
            network_demand - demand obtained from the network in TWh
            eia_historic_demand - demand obtained from EIA in TWh
            eia_ember_demands - demand obtained from Ember in TWh
        Output:
            demand - DataFrame comparing two data
    """
    demand = pd.DataFrame(columns=["PyPSA Model", "EIA", "Ember"])
    demand.loc["Electricity Demand [TWh]", ["PyPSA Model", "EIA", "Ember"]] = [network_demand, eia_historic_demand, ember_historic_demand]
    demand["Error wrt. EIA (%)"] = (100*(demand["PyPSA Model"] - demand["EIA"])/demand["EIA"]).astype(float)
    demand["Error wrt Ember (%)"] = (100*(demand["PyPSA Model"] - demand["Ember"])/demand["Ember"]).astype(float)
    return demand.round(2)


def compare_generation(network_generation, eia_historic_generation, ember_historic_generation):
    """ Input:
            network_generation - electricity generation obtained from the network in TWh
            eia_historic_generation - electricity generation obtained from EIA in TWh
            ember_historic_generation - electricity generation obtained from Ember in TWh
        Output:
            generation - DataFrame comparing two data
    """
    # bring historic data into format
    eia_historic_generation["country"] = eia_historic_generation.country.str.replace("(billion kWh)","").str.strip()
    eia_historic_generation.set_index("country", inplace=True)
    eia_historic_generation.columns = ["EIA"]
    eia_historic_generation = eia_historic_generation.astype(float)
    eia_historic_generation.rename(index={"Generation":"Total generation", 
                                          "Hydroelectricity":"Hydro", 
                                          "Biomass and waste":"Biomass", 
                                          "Hydroelectric pumped storage":"PHS"}, inplace=True)
    eia_historic_generation.drop(index=["Fossil fuels", "Renewables", "Non-hydroelectric renewables", 
                                        "Solar, tide, wave, fuel cell", "Tide and wave"], inplace=True)
    eia_historic_generation.loc["Load shedding"] = None
    eia_historic_generation.loc["Natural gas"] = eia_historic_generation.loc[["Natural gas", "Other gases"]].sum()
    eia_historic_generation = eia_historic_generation.loc[["Nuclear", "Coal", "Natural gas", "Oil", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Geothermal", "Load shedding", "Total generation"], :]

    # bring generation from Ember dataset into format
    ember_historic_generation.columns = ["Ember"]
    ember_historic_generation.loc["Total generation",:] = ember_historic_generation.sum()
    
    # bring generation from the network into format
    all_carriers = ["nuclear", "coal", "lignite", "CCGT", "OCGT", "oil", "hydro", "ror", "PHS", "solar", "offwind-ac", "offwind-dc", "onwind", "biomass", "geothermal", "load"]
    network_generation = network_generation.reindex(all_carriers, fill_value=0)
    network_generation.rename(index={"nuclear":"Nuclear", "oil":"Oil", "solar":"Solar", "biomass":"Biomass", "geothermal":"Geothermal", "load":"Load shedding"}, inplace=True)
    network_generation["Coal"] = network_generation[["coal", "lignite"]].sum()
    network_generation["Natural gas"] = network_generation[["CCGT", "OCGT"]].sum()
    network_generation["Hydro"] = network_generation[["hydro", "ror"]].sum()
    network_generation["Wind"] = network_generation[["offwind-ac", "offwind-dc", "onwind"]].sum()
    network_generation["Load shedding"] /= 1e3
    network_generation = network_generation.loc[["Nuclear", "Coal", "Natural gas", "Oil", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Geothermal", "Load shedding"]]
    network_generation["Total generation"] = network_generation.sum()
    network_generation.name = "PyPSA Model"
    network_generation = network_generation.to_frame()

    # merge two data
    generation = pd.concat([network_generation, eia_historic_generation, ember_historic_generation], axis=1).astype(float)
    generation["Error wrt EIA (%)"] = (100*(generation["PyPSA Model"] - generation["EIA"])/generation["EIA"]).astype(float).round(2)
    generation["Error wrt Ember (%)"] = (100*(generation["PyPSA Model"] - generation["Ember"])/generation["Ember"]).astype(float).round(2)
    generation.fillna(0, inplace=True)
    generation = generation.loc[["Nuclear", "Coal", "Natural gas", "Oil", "Hydro", "PHS", "Solar", "Wind", "Biomass", "Geothermal", "Load shedding", "Total generation"], :].round(2)
    return generation


def compare_network_lines(network_length, network_voltages, real_length, real_voltages):
    """ Input:
            network_length - network length in km
            network_voltages - network voltages in kV
            real_length - real network length in km
            real_voltages - real network voltages in kV
        Output:
            df_network - DataFrame comparing two data
    """
    voltage_ratings = sorted(list(set(network_voltages) | set(real_voltages)))
    df_network = pd.DataFrame(index=["Line length [km]"]+voltage_ratings)
    df_network.loc["Line length [km]", ["PyPSA Model", "Cited Sources"]] = [network_length, real_length]
    df_network.loc["Line length [km]", "Error (%)"] = (100*(network_length - real_length)/real_length).astype(float).round(2)
    df_network.loc[network_voltages, "PyPSA Model"] = "+"
    df_network.loc[real_voltages, "Cited Sources"] = "+"
    return df_network

In [ ]:
# directory to data and result folders
DATA_DIR = os.path.join(os.getcwd(), "../data/")
RESULTS_DIR = os.path.join(os.getcwd(), "../pypsa_data/results")
BASE_DIR = os.path.join(os.getcwd(), "../pypsa_data/networks")
OUTPUT_DIR = os.path.join(os.getcwd(), "../results/")

# create results folder
create_folder(OUTPUT_DIR)

# get list of countries
country_code_list = get_country_list(RESULTS_DIR)

# list of horizons
horizons = [2021]

for horizon in horizons:
    with pd.ExcelWriter(OUTPUT_DIR+f'JI-GIS_country_validation_{horizon}.xlsx', engine='xlsxwriter') as writer:
        for country_code in country_code_list:
            # get filepath of the network
            filepath = get_network_path(country_code, horizon, RESULTS_DIR)
            # get filepath to base network
            base_filepath = get_base_network_path(country_code, horizon, BASE_DIR)
            
            # load the network
            n = load_network(filepath)
            # load base network
            n_base = load_network(base_filepath)
    
            # get total electricity demand
            demand = get_total_demand(n)
    
            # get optimal generation capacities
            generation_capacities = get_installed_capacities(n)
    
            # get generation mix
            generation_mix = get_generation_mix(n)

            # get network length and voltage ratings
            network_length, network_voltages = get_network_length(n_base)
    
            # get electricity consumption of the country from EIA dataset
            eia_historic_demand = get_eia_historic_demand(country_code, horizon)
            # get electricity consumption of country from Ember dataset
            ember_historic_demand = get_ember_historic_demand(country_code, horizon)
    
            # get historic generation capacities of the country from EIA dataset
            eia_historic_capacities = get_eia_historic_capacities(country_code, horizon)
            # get historic generation capacities of the country from Ember dataset
            ember_historic_capacities = get_ember_historic_capacities(country_code, horizon)

            # get EIA historic generation mix of the country
            eia_historic_generation = get_eia_historic_generation(country_code, horizon)
            # get Ember historic generation mix of the country
            ember_historic_generation = get_ember_historic_generation(country_code, horizon)

            # get real network length and voltages
            real_length, real_voltages = real_network_length(country_code)
    
            # compare the data
            demand_comparison = compare_demands(demand, eia_historic_demand, ember_historic_demand)
            capacity_comparison = compare_capacities(generation_capacities, eia_historic_capacities, ember_historic_capacities)
            generation_comparison = compare_generation(generation_mix, eia_historic_generation, ember_historic_generation)
            network_comparison = compare_network_lines(network_length, network_voltages, real_length, real_voltages)

            demand_comparison.to_excel(writer, sheet_name=country_code, startrow=0, startcol=0)
            capacity_comparison.to_excel(writer, sheet_name=country_code, startrow=3, startcol=0)
            generation_comparison.to_excel(writer, sheet_name=country_code, startrow=0, startcol=7)
            network_comparison.to_excel(writer, sheet_name=country_code, startrow=0, startcol=14)
